# FLARE Query Engine

Adapted from the paper "Active Retrieval Augmented Generation"

Currently implements FLARE Instruct, which tells the LLM to generate retrieval instructions.

In [1]:
import os
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from llama_index.query_engine import FLAREInstructQueryEngine
from llama_index import (
    VectorStoreIndex,
    ResponseSynthesizer,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    LLMPredictor,
    ServiceContext,
)

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.query_engine.flare_query_engine import DEFAULT_INSTRUCT_PROMPT_TMPL
print(DEFAULT_INSTRUCT_PROMPT_TMPL)

Skill 1. Use the Search API to look up relevant information by writing "[Search(query)]" where "query" is the search query you want to look up. For example: 

Query: But what are the risks during production of nanomaterials?
Answer: [Search(What are some nanomaterial production risks?)]

Query: The colors on the flag of Ghana have the following meanings.
Answer: Red is for [Search(What is the meaning of Ghana's flag being red?)], green for forests, and gold for mineral wealth.

Query: What did the author do during his time in college?
Answer: The author took introductory CS classes during his freshman year. He also took some humanities classes, such as [Search(What humanities classes did the author take in college?)].




Now given the query, please answer the following question. You may use the Search API "[Search(query)]" whenever possible.
If the answer is complete and no longer contains any "[Search(query)]" tags, write "done" to finish the task.


Query: {query_str}
Answer: {exist

In [3]:
service_context = ServiceContext.from_defaults(
    llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)),
    chunk_size=512
)

In [4]:
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [5]:
index_query_engine = index.as_query_engine(similarity_top_k=2)

In [6]:
flare_query_engine = FLAREInstructQueryEngine(
    query_engine=index_query_engine,
    service_context=service_context,
    verbose=True
)

In [7]:
flare_query_engine

In [8]:
response = flare_query_engine.query("What did the author do during his time in college?")

Query: What did the author do during his time in college?
Current response: 
Skill 1. Use the Search API to look up relevant information by writing "[Search(query)]" where "query" is the search query you want to look up. For example: 

Query: But what are the risks during production of nanomaterials?
Answer: [Search(What are some nanomaterial production risks?)]

Query: The colors on the flag of Ghana have the following meanings.
Answer: Red is for [Search(What is the meaning of Ghana's flag being red?)], green for forests, and gold for mineral wealth.

Query: What did the author do during his time in college?
Answer: The author took introductory CS classes during his freshman year. He also took some humanities classes, such as [Search(What humanities classes did the author take in college?)].




Now given the query, please answer the following question. You may use the Search API "[Search(query)]" whenever possible.
If the answer is complete and no longer contains any "[Search(query)